In [ ]:
import numpy as np, matplotlib.pyplot as plt
from sympy.geometry import Point, convex_hull
from scipy.spatial import ConvexHull

%matplotlib inline

def read_poly(file_name):
    """
    Simple poly-file reader, that creates a python dictionary 
    with information about vertices, edges and holes.
    It assumes that vertices have no attributes or boundary markers.
    It assumes that edges have no boundary markers.
    No regional attributes or area constraints are parsed.
    """
    
    output = {'vertices': None, 'holes': None, 'segments': None}

    # open file and store lines in a list
    file = open(file_name, 'r')
    lines = file.readlines()
    file.close()
    lines = [x.strip('\n').split() for x in lines]

    # Store vertices
    vertices= []
    N_vertices, dimension, attr, bdry_markers = [int(x) for x in lines[0]]
    # We assume attr = bdrt_markers = 0
    for k in range(N_vertices):
        label, x, y = [items for items in lines[k+1]]
        vertices.append([float(x), float(y)])
        
    output['vertices'] = np.array(vertices)

    # Store segments
    segments = []
    N_segments, bdry_markers = [int(x) for x in lines[N_vertices+1]]
    for k in range(N_segments):
        label, pointer_1, pointer_2 = [items for items in lines[N_vertices+k+2]]
        segments.append([int(pointer_1)-1, int(pointer_2)-1])

    output['segments'] = np.array(segments)

    # Store holes
    N_holes = int(lines[N_segments+N_vertices+2][0])
    holes = []
    for k in range(N_holes):
        label, x, y = [items for items in lines[N_segments + N_vertices + 3 + k]]
        holes.append([float(x), float(y)])

    output['holes'] = np.array(holes)

    return output

In [ ]:
lake_superior = read_poly("superior.poly")

vertices_ls = lake_superior['vertices']

%time hull = ConvexHull(vertices_ls)

In [ ]:
vertices_sympy = [Point(x) for x in vertices_ls]

%time convex_hull(*vertices_sympy)

In [ ]:
plt.figure(figsize=(12,6))
plt.xlim(vertices_ls[:,0].min()-0.01, vertices_ls[:,0].max()+0.01)
plt.ylim(vertices_ls[:,1].min()-0.01, vertices_ls[:,1].max()+0.01)
plt.axis('off')
plt.axes().set_aspect('equal')
plt.plot(vertices_ls[:,0], vertices_ls[:,1], 'b.')

for simplex in hull.simplices:
    plt.plot(vertices_ls[simplex, 0], vertices_ls[simplex, 1], 'r-')

plt.show()

In [ ]:
points = np.random.rand(320, 3)

hull = ConvexHull(points)

X = hull.points[:, 0]
Y = hull.points[:, 1]
Z = hull.points[:, 2]

from mayavi import mlab

mlab.triangular_mesh(X, Y, X, hull.simplices, colormap='gray', opacity=0.5, representation='wireframe')

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d

vor = Voronoi(vertices_ls)

In [ ]:
plt.figure(figsize=(12,6))
ax = plt.subplot(111, aspect='equal')
voronoi_plot_2d(vor, ax=ax)
plt.xlim( 0.45,  0.50)
plt.ylim(-0.40, -0.35)
plt.show()

In [ ]:
vor.point_region

In [ ]:
vor.vertices

In [ ]:
[vor.vertices[x] for x in vor.regions[vor.point_region[4]]]

In [ ]:
vor.ridge_points

In [ ]:
from scipy.spatial import Delaunay

tri = Delaunay(vertices_ls)

plt.figure(figsize=(12,6))
plt.xlim(vertices_ls[:,0].min()-0.01, vertices_ls[:,0].max()+0.01)
plt.ylim(vertices_ls[:,1].min()-0.01, vertices_ls[:,1].max()+0.01)
plt.axes().set_aspect('equal')
plt.axis('off')
plt.triplot(vertices_ls[:,0], vertices_ls[:,1], tri.simplices, 'k-')
plt.plot(vertices_ls[:,0], vertices_ls[:,1], 'r.')
plt.show()

In [ ]:
from triangle import triangulate, plot as tplot

cndt = triangulate(lake_superior, 'p')

plt.figure(figsize=(24,12))
ax = plt.subplot(111, aspect='equal')
tplot.plot(ax, **cndt)
plt.show()

In [ ]:
cfdt = triangulate(lake_superior, 'pD')

In [ ]:
cncfq20dt = triangulate(lake_superior, 'pq20D')

plt.figure(figsize=(24,12))
ax = plt.subplot(111, aspect='equal')
tplot.plot(ax, **cncfq20dt)
plt.show()

In [ ]:
cncfq20adt = triangulate(lake_superior, 'pq20a.001D')

plt.figure(figsize=(24,12))
ax = plt.subplot(111, aspect='equal')
tplot.plot(ax, **cncfq20adt)
plt.show()

In [ ]:
X = cncfq20adt['triangles'][:,0]
Y = cncfq20adt['triangles'][:,1]
Z = cncfq20adt['triangles'][:,2]

Xvert = [cncfq20adt['vertices'][x] for x in X]
Yvert = [cncfq20adt['vertices'][y] for y in Y]
Zvert = [cncfq20adt['vertices'][z] for z in Z]

from scipy.spatial import minkowski_distance

lengthsXY = minkowski_distance(Xvert, Yvert)
lengthsXZ = minkowski_distance(Xvert, Zvert)
lengthsYZ = minkowski_distance(Yvert, Zvert)

In [ ]:
from scipy.sparse import lil_matrix
from scipy.sparse.csgraph import shortest_path

nvert = len(cncfq20adt['vertices'])
G = lil_matrix((nvert, nvert))

for k in range(len(X)):
    G[X[k], Y[k]] = G[Y[k], X[k]] = lengthsXY[k]
    G[X[k], Z[k]] = G[Z[k], X[k]] = lengthsXZ[k]
    G[Y[k], Z[k]] = G[Z[k], Y[k]] = lengthsYZ[k]
    
dist_matrix, pred = shortest_path(G, return_predecessors=True, directed=True, unweighted=False)

index = 370
path = [370]

while index != 197:
    index = pred[197, index]
    path.append(index)

print path

In [ ]:
plt.figure(figsize=(24,12))
ax = plt.subplot(111, aspect='equal')
tplot.plot(ax, **cncfq20adt)

Xs = [cncfq20adt['vertices'][x][0] for x in path]
Ys = [cncfq20adt['vertices'][x][1] for x in path]

ax.plot(Xs, Ys, '-', linewidth=5, color='blue')
plt.show()